Identical codes to the evictions outside of covid. Merged with BBL datasets and saved to cloud.

In [3]:
# !pip install geopandas folium matplotlib seaborn scipy
# !pip install esda
# !pip install splot
# # for google colab, had to reinstall some pacakges.

In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import scipy

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist

# visualization
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
import folium
from folium.plugins import HeatMap
from folium import Marker
from folium.plugins import MarkerCluster
import plotly.express as px
import plotly.io as pio

# spatial statistics
from esda.moran import Moran
from esda.getisord import G_Local
from libpysal.weights import Queen, Rook

# system and utility
import warnings
import os
import io
from IPython.display import IFrame
from google.colab import files

from libpysal.weights import Queen, Rook
from esda.moran import Moran
import matplotlib.pyplot as plt
from splot.esda import moran_scatterplot

# suppress warnings
warnings.filterwarnings('ignore')

# inline
%matplotlib inline

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# data source:
file_path1 = '/content/drive/My Drive/X999/evictions_pre_post_covid.csv'
file_path2 = '/content/drive/My Drive/X999/evictions_covid.csv'
file_path3 = '/content/drive/My Drive/X999/bbl_cleaned.csv'

In [7]:
evictions_pre_post_raw = pd.read_csv(file_path1)
evictions_covid_raw = pd.read_csv(file_path2)

In [8]:
evictions_pre_post = evictions_pre_post_raw.copy()
evictions_covid = evictions_covid_raw.copy()

In [ ]:
# evictions_pre_post.columns, \
# evictions_covid.shape

In [9]:
bbl = pd.read_csv(file_path3)

In [10]:
bbl_df = bbl.copy()

In [11]:
len(list(bbl_df.columns))
# correct

112

In [12]:
bbl_df.head(4)

,borough,block,lot,community board,census tract 2010,cb2010,schooldist,council district,postcode,firecomp,...,floor_category,building_age,decade,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile
0,BK,8366,222,318.0,696.02,2002.0,22.0,46.0,11234.0,E323,...,low-rise,6.0,2010-2019,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",single-unit,True,very small,Q1 (smallest 25%)
1,SI,2088,7501,502.0,273.01,1003.0,31.0,50.0,10314.0,E166,...,low-rise,2025.0,Pre-1900,"Pre-1947, pre-rent-control",Pre-1900,"Pre-1929, pre-great depression",6-20 units,False,very large,Q4 (largest 25%)
2,QN,15932,7501,414.0,964.00,2015.0,27.0,31.0,11692.0,L121,...,low-rise,2025.0,Pre-1900,"Pre-1947, pre-rent-control",Pre-1900,"Pre-1929, pre-great depression",6-20 units,False,large,Q4 (largest 25%)
3,BK,2571,28,301.0,561.00,1005.0,14.0,33.0,11222.0,L106,...,low-rise,7.0,2010-2019,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",single-unit,True,medium-small,Q4 (largest 25%)


In [ ]:
# list(bbl_df.columns)

In [13]:
evictions_pre_post.columns

Index(['court_index_number', 'docket_number', 'eviction_address',
       'eviction_apartment_number', 'executed_date', 'borough',
       'eviction_postcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'bbl', 'nta', 'geometry', 'eviction_count',
       'year', 'average_year_eviction_count'],
      dtype='object')

In [14]:
# only check the relevant ones
bbl_cleaned = bbl_df[['bbl', 'yearbuilt', 'bldgclass', 'numfloors', 'unitsres', 'ownername', 'bldgarea','building_type', 'building_category', 'is_condo', 'floor_category',
           'rent_era', 'architectural_style', 'economic_period',
           'residential_units_category', 'is_llc', 'building_size_category', 'size_quartile', 'decade']]
bbl_cleaned.head()

,bbl,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade
0,3083660222,2019.0,A5,2.0,1.0,"EAST 69 AVENUE N DEVELOPMENT, LLC",1288.0,post-war,single-family,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",single-unit,True,very small,Q1 (smallest 25%),2010-2019
1,5020887501,0.0,R3,0.0,16.0,BAYBERRY WOODS CONDOMINIUM,26400.0,pre-war,condo-co-op,True,low-rise,"Pre-1947, pre-rent-control",Pre-1900,"Pre-1929, pre-great depression",6-20 units,False,very large,Q4 (largest 25%),Pre-1900
2,4159327501,0.0,R3,0.0,18.0,UNAVAILABLE OWNER,16599.0,pre-war,condo-co-op,True,low-rise,"Pre-1947, pre-rent-control",Pre-1900,"Pre-1929, pre-great depression",6-20 units,False,large,Q4 (largest 25%),Pre-1900
3,3025710028,2018.0,A5,3.0,1.0,85 CALYER STREET LLC,3478.0,post-war,single-family,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",single-unit,True,medium-small,Q4 (largest 25%),2010-2019
4,3015147501,0.0,R2,0.0,12.0,UNAVAILABLE OWNER,11134.0,pre-war,condo-co-op,True,low-rise,"Pre-1947, pre-rent-control",Pre-1900,"Pre-1929, pre-great depression",6-20 units,False,large,Q4 (largest 25%),Pre-1900


In [15]:
len(bbl_cleaned)

753110

In [23]:
type(evictions_covid['bbl'][0]), type(bbl_cleaned['bbl'][0])

(numpy.float64, str)

In [24]:
# convert both to strings
evictions_covid['bbl'] = evictions_covid['bbl'].astype(str)
bbl_cleaned['bbl'] = bbl_cleaned['bbl'].astype(str)
type(evictions_covid['bbl'][0]), type(bbl_cleaned['bbl'][0])

(str, str)

In [25]:
# use set to check common ones
eviction_bbls = set(evictions_covid['bbl'])
building_bbls = set(bbl_cleaned['bbl'])
common_bbls = eviction_bbls.intersection(building_bbls)
print(f"evictions data unique bbls: {len(eviction_bbls)}")
print(f"building data unqiue bbls: {len(building_bbls)}")
print(f"number of common bbls: {len(common_bbls)}")
print(f"percentage of eviction bbls found in building data: {len(common_bbls)/len(eviction_bbls)*100:.2f}%")
# so, at first, there were no common ones

evictions data unique bbls: 4978
building data unqiue bbls: 753110
number of common bbls: 0
percentage of eviction bbls found in building data: 0.00%


In [26]:
print("eveiction bbls: ", evictions_covid['bbl'].head(10).tolist())
print("building bbls: ", bbl_cleaned['bbl'].head(10).tolist())
# 10 and 8/9 digits
# and evictions data has digits

eveiction bbls:  ['2028200035.0', '1016447502.0', '4031500075.0', '2037760056.0', '4007200032.0', '1021360090.0', '1021730015.0', '4058937501.0', '1022270001.0', '2028410070.0']
building bbls:  ['3083660222', '5020887501', '4159327501', '3025710028', '3015147501', '3015057501', '5054397501', '4010077501', '3031970008', '5020877501']


In [27]:
print("evictions BBL string length:", \
      evictions_covid['bbl'].str.len().value_counts())
print("building BBL string length:", \
      bbl_cleaned['bbl'].str.len().value_counts())
# so, indivisual lengths and total counts are different

evictions BBL string length: bbl
12    6338
Name: count, dtype: int64
building BBL string length: bbl
10    753110
Name: count, dtype: int64


In [28]:
def clean_bbl(bbl_val):
    if pd.isna(bbl_val):
        return None
    bbl_as_string = str(bbl_val)
    digits_only = ""
    for character in bbl_as_string:
        if character.isdigit():
            digits_only = digits_only + character
    first_ten_digits = digits_only[:10]
    final_bbl = first_ten_digits.zfill(10)
    return final_bbl

In [34]:
evictions_covid['bbl_clean'] = evictions_covid['bbl'].apply(clean_bbl)
bbl_cleaned['bbl_clean'] = bbl_cleaned['bbl'].apply(clean_bbl)

eviction_bbls_clean = set(evictions_covid['bbl_clean'].dropna())
building_bbls_clean = set(bbl_cleaned['bbl_clean'].dropna())
common_bbls_clean = eviction_bbls_clean.intersection(building_bbls_clean)

f"number of common BBLs after thorough cleaning: {len(common_bbls_clean)}", len(bbl_cleaned), len(evictions_pre_post), \
len(set(evictions_covid['bbl'])), \
len(set(bbl_cleaned['bbl'])), \
4978/6338 # 78.54% matched bbl in eviction data

('number of common BBLs after thorough cleaning: 4753',
 753110,
 74082,
 4978,
 753110,
 0.7854212685389713)

In [35]:
eviction_bbls_std = set(evictions_covid['bbl_clean'])
building_bbls_std = set(bbl_cleaned['bbl_clean'])
len(evictions_covid['bbl']), \
len(evictions_covid['bbl_clean']), \
len(bbl_cleaned['bbl_clean']), \
len(bbl_cleaned['bbl']), \
len(eviction_bbls_std), \
len(building_bbls_std)
# 32856 should be the number we aim for common bbls

(6338, 6338, 753110, 753110, 4978, 753110)

In [36]:
evictions_covid['bbl_clean'] = evictions_covid['bbl_clean'].astype(str).str.strip()
bbl_cleaned['bbl_clean'] = bbl_cleaned['bbl_clean'].astype(str).str.strip()

In [37]:
eviction_bbls_std = set(evictions_covid['bbl_clean'])
building_bbls_std = set(bbl_cleaned['bbl_clean'])
common_bbls_std = eviction_bbls_std.intersection(building_bbls_std)
print(f"number of common bbl after standardization: {len(common_bbls_std)}")

number of common bbl after standardization: 4753


In [38]:
cleaned_merge = pd.merge(
    evictions_covid,
    bbl_cleaned[['bbl_clean', 'yearbuilt', 'bldgclass', 'numfloors', 'unitsres', 'ownername', 'bldgarea','building_type', 'building_category', 'is_condo', 'floor_category',
           'rent_era', 'architectural_style', 'economic_period',
           'residential_units_category', 'is_llc', 'building_size_category', 'size_quartile', 'decade']],
    on='bbl_clean',
    how='left'
)

f"number of non null building_type values in cleaned merge: {cleaned_merge['building_type'].notna().sum()}", \
len(cleaned_merge), len(bbl_cleaned), len(evictions_pre_post)
# there are way more buildings than the ones that had the evictions.

('number of non null building_type values in cleaned merge: 6024',
 6338,
 753110,
 74082)

In [39]:
display(cleaned_merge.head())
# shows all the columns

,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,eviction_postcode,ejectment,eviction/legal_possession,latitude,...,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade
0,35484/19,494073,184 MT. EDEN PARKWAY,UNIT 1D,2020-01-02,BRONX,10457,Not an Ejectment,Possession,40.842943,...,False,mid-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","Pre-1929, pre-great depression",21-100 units,True,large,Q4 (largest 25%),1920-1929
1,251388/19,117473,160 EAST 117 STREET,3-B,2020-01-02,MANHATTAN,10035,Not an Ejectment,Possession,40.799094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66822/19,25733,66-07 ALDERTON ST,unknown,2020-01-02,QUEENS,11374,Not an Ejectment,Possession,40.719316,...,False,low-rise,"1994–Present, vacancy decontrol","1981–2000, Post-Modernism","1991–2008, modern economic growth",3-5 units,True,small,Q4 (largest 25%),1990-1999
3,68501/18,91505,1245 STRATFORD AVE,D12,2020-01-02,BRONX,10472,Not an Ejectment,Possession,40.830623,...,False,mid-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","Pre-1929, pre-great depression",21-100 units,True,very large,Q4 (largest 25%),1920-1929
4,68498/19,26147,28-16 47TH STREET,1-L,2020-01-02,QUEENS,11103,Not an Ejectment,Possession,40.762390,...,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","Pre-1929, pre-great depression",6-20 units,True,medium,Q4 (largest 25%),1920-1929


In [40]:
cleaned_merge.to_csv('/content/drive/My Drive/X999/bbl_evictions_merged_covid.csv', index=False)

In [42]:
file_path1 = '/content/drive/My Drive/X999/bbl_evictions_merged_covid.csv'

In [43]:
df = pd.read_csv(file_path1)

In [44]:
df.columns

Index(['court_index_number', 'docket_number', 'eviction_address',
       'eviction_apartment_number', 'executed_date', 'borough',
       'eviction_postcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'bbl', 'nta', 'geometry', 'eviction_count',
       'year', 'average_year_eviction_count', 'bbl_clean', 'yearbuilt',
       'bldgclass', 'numfloors', 'unitsres', 'ownername', 'bldgarea',
       'building_type', 'building_category', 'is_condo', 'floor_category',
       'rent_era', 'architectural_style', 'economic_period',
       'residential_units_category', 'is_llc', 'building_size_category',
       'size_quartile', 'decade'],
      dtype='object')

In [ ]:
# decade added